In [2]:
import pandas as pd
import numpy as np

In [ ]:
for year in ('2017','2018','2019'):
    weather_df = pd.read_csv(f'./snow_data/{year}asos.txt')
    weather_df.rename({'valid':'datetime'},axis=1,inplace=True)

    # Let 'GW' denote Good Weather (aka normal weather)
    weather_df['wxcodes'] = weather_df.wxcodes.replace(np.NaN, 'GW')
    np.unique(weather_df.wxcodes)

    def is_intense_snow(s):
        if '+' in s or '-' in s:
            if 'SN' in s:
                return 1
        return 0

    def is_snow(s):
        if '+' not in s or '-' not in s:    
            if 'SN' in s:
                return 1
        return 0

    weather_df['heavy_snow'] = weather_df['wxcodes'].apply(lambda x: is_intense_snow(x))
    weather_df['snow'] = weather_df['wxcodes'].apply(lambda x: is_snow(x))

    weather_df['datetime'] = pd.to_datetime(weather_df.datetime).dt.round('H')

    # Drop rows with missing temperatures
    missing_weather = weather_df.loc[weather_df.tmpc.isna()].index
    weather_df.drop(missing_weather, inplace=True)

    weather_df = weather_df.groupby('datetime').agg({'tmpc': np.max, 'heavy_snow': np.sum, 'snow': np.sum}).reset_index()

    weather_df.to_csv(f'./data/{year}weather.csv')
    del weather_df

Weather data:
- Replaced missing values with `'GW'` as default
- Only took into account `snow`, since rain was deemed not important from the first assignment (subset)
- Converted to `datetime` format, and rounded to the closest hour to merge (minute accuracy is not important)
- Groupby `datetime` since we needed to take the average reading of the 4 weather stations
- We take the **max** temperature to denote the "worst case" temperature of the day
- And the snow metrics are the total sum of warnings during that hour (more snow warnings = more snowfall)

In [4]:
for year in ('2017','2018'):
    output = pd.DataFrame()
    for i in ('01','02','03','04','11','12'):
        print(f"Processing {i}")
        sample_data = pd.read_csv(f'./snow_data/yellow_tripdata_{year}-{i}.csv')

        # Ensure all values fall within the defined values in the Dictionary specified
        sample_data['RatecodeID'] = sample_data['RatecodeID'].apply(lambda x: int(x) if x in range(1, 7) else np.NaN)
        sample_data['payment_type'] = sample_data['payment_type'].apply(lambda x: int(x) if x in range(1, 7) else np.NaN)

        # Convert to datetime
        sample_data['PUdatetime'] = pd.to_datetime(sample_data['tpep_pickup_datetime'])
        sample_data['DOdatetime'] = pd.to_datetime(sample_data['tpep_dropoff_datetime'])

        # Add trip duration by minutes
        sample_data['trip_duration'] = sample_data['DOdatetime'] - sample_data['PUdatetime']
        sample_data['trip_duration'] = sample_data['trip_duration'].dt.seconds // 60 # round to the nearest minute so we preserve int

        sample_data.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'],axis=1,inplace=True)

        # This time, we exclude all non-positive trips (loss of money) since we are working with profitability, and
        # the non-positive trips do not follow any known distribution
        invalid_vals = sample_data[(sample_data[['trip_duration',
                                                 'trip_distance','fare_amount','total_amount',
                                                'extra']] <= 0).any(axis=1)].index

        invalid_val_distribution = sample_data.loc[invalid_vals]

        sample_data = sample_data.drop(axis=0, index=invalid_vals)
        sample_data = sample_data.dropna()


        output = pd.concat([output, sample_data])

        del sample_data

    weather = pd.read_csv(f'./data/{year}weather.csv', index_col=0)

    weather['datetime'] = pd.to_datetime(weather['datetime'])

    output['dow'] = output['PUdatetime'].dt.dayofweek
    output['hour'] = output['PUdatetime'].dt.hour
    output['datetime'] = output['PUdatetime'].dt.round('H')
    output.drop(['PUdatetime','DOdatetime','improvement_surcharge'], axis=1, inplace=True)

    output = pd.merge(output, weather, on='datetime', how='left').drop('datetime', axis=1)

    output.reset_index().to_feather(f'./data/{year}taxi_weather.feather')

Processing 01
Processing 02
Processing 03
Processing 04
Processing 11
Processing 12
Processing 01
Processing 02
Processing 03
Processing 04
Processing 11
Processing 12


Taxi Data:
- Firstly, remove all outright invalid instances according to the data dictionary
- Convert to datetime
- Add duration in minutes (Refer to ETL of first assignment)
- **But**, we now remove all negative values / disputed trips. This is because we want to work with the profitability of each trip, not the *loss* of profitability 
    - i.e We don't want to be predicting the probability of a trip being disputed / no charge
    - It means we can remove inconsistencies
- We may consider a `datetime` model, but suggest that we use `dow` and `hour`
- We take note that `improvement_surcharge` is a constant 30 cents, so we can remove the attribute with ease

In [ ]:
year = "2019"
output = pd.DataFrame()
for i in ('01','02','03','04'):
    print(f"Processing {i}")
    sample_data = pd.read_csv(f'./snow_data/yellow_tripdata_{year}-{i}.csv')

    # Ensure all values fall within the defined values in the Dictionary specified
    sample_data['RatecodeID'] = sample_data['RatecodeID'].apply(lambda x: int(x) if x in range(1, 7) else np.NaN)
    sample_data['payment_type'] = sample_data['payment_type'].apply(lambda x: int(x) if x in range(1, 7) else np.NaN)

    # Convert to datetime
    sample_data['PUdatetime'] = pd.to_datetime(sample_data['tpep_pickup_datetime'])
    sample_data['DOdatetime'] = pd.to_datetime(sample_data['tpep_dropoff_datetime'])

    # Add trip duration by minutes
    sample_data['trip_duration'] = sample_data['DOdatetime'] - sample_data['PUdatetime']
    sample_data['trip_duration'] = sample_data['trip_duration'].dt.seconds // 60 # round to the nearest minute so we preserve int

    sample_data.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'],axis=1,inplace=True)

    # This time, we exclude all non-positive trips (loss of money) since we are working with profitability, and
    # the non-positive trips do not follow any known distribution
    invalid_vals = sample_data[(sample_data[['trip_duration',
                                             'trip_distance','fare_amount','total_amount',
                                            'extra']] <= 0).any(axis=1)].index

    invalid_val_distribution = sample_data.loc[invalid_vals]

    sample_data = sample_data.drop(axis=0, index=invalid_vals)
    sample_data = sample_data.dropna()


    output = pd.concat([output, sample_data])

    del sample_data

weather = pd.read_csv(f'./data/{year}weather.csv', index_col=0)

weather['datetime'] = pd.to_datetime(weather['datetime'])

output['dow'] = output['PUdatetime'].dt.dayofweek
output['hour'] = output['PUdatetime'].dt.hour
output['datetime'] = output['PUdatetime'].dt.round('H')
output.drop(['PUdatetime','DOdatetime','improvement_surcharge'], axis=1, inplace=True)

output = pd.merge(output, weather, on='datetime', how='left').drop('datetime', axis=1)

to_drop = output.loc[output['RatecodeID'] == 5].index

to_drop = to_drop.append(output.loc[output['RatecodeID'] == 6].index)
to_drop = to_drop.append(output.loc[output['payment_type'] == 3].index)
to_drop = to_drop.append(output.loc[output['payment_type'] == 4].index)
to_drop = to_drop.append(output.loc[output['payment_type'] == 5].index)
to_drop = to_drop.append(output.loc[output['payment_type'] == 6].index)
to_drop = to_drop.append(output.loc[output['fare_amount'] >= 500].index)
to_drop = to_drop.append(output.loc[output['tolls_amount'] > 374.94].index)
to_drop = to_drop.append(output.loc[output['total_amount'] <= 2.5].index)    
output.drop(index=to_drop, axis=1, inplace=True)

output = output[["PULocationID","hour", "dow","tmpc","heavy_snow","snow"]]

output.reset_index().to_feather(f'./ml_data/{year}test.feather')